# Ejercicio 3 –Spark SQL: consultas

In [1]:
# Import findspark
import findspark

# Configure the environment
findspark.init()

# Import the Spark components required for the session creation
from pyspark import SparkConf
from pyspark.sql import SparkSession

# Configure and create the session
conf = SparkConf()
conf = conf.setAppName('mds-session')
conf = conf.setMaster('local[*]')
spark = SparkSession.builder.config(conf = conf).getOrCreate()

22/04/04 19:58:59 WARN Utils: Your hostname, mdsuser resolves to a loopback address: 127.0.1.1; using 192.168.242.129 instead (on interface ens33)
22/04/04 19:58:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/04 19:59:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/04 19:59:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/04 19:59:04 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
# load albums
album_df = spark.read.options(sep='\t', header=False, inferSchema=True).csv('./data/albums.tsv')
album_df = album_df.toDF('id', 'title')
album_df.createOrReplaceTempView('album')
spark.sql('SELECT * FROM album').show(5)

+------+--------------------+
|    id|               title|
+------+--------------------+
|300848|         Fear Itself|
|300822|          Dimensions|
|514953|Las Numero 1 De L...|
|287650|       Friend Or Foe|
|611336|       Muertos Vivos|
+------+--------------------+
only showing top 5 rows



In [3]:
# load artists
artist_df = spark.read.options(sep='\t', header=False, inferSchema=True).csv('./data/artists.tsv')
artist_df = artist_df.toDF('id', 'name', 'hotness', 'familiarity', 'location')
artist_df.createOrReplaceTempView('artist')
spark.sql('SELECT * FROM artist').show(5)

+------------------+----------------+-----------+-----------+------------------+
|                id|            name|    hotness|familiarity|          location|
+------------------+----------------+-----------+-----------+------------------+
|AR009211187B989185|Carroll Thompson|0.297404803|0.396817844|     Not available|
|AR00A6H1187FB5402A|     The Meatmen|0.395627514|0.514064276|       Lansing, MI|
|AR00LNI1187FB444A5|    Bruce BecVar|0.340275507|0.471212607|     Not available|
|AR00MBZ1187B9B5DB1|  Memphis Minnie|0.360644227|0.588571842|       Algiers, LA|
|AR01IP11187B9AF5D2|Call To Preserve|0.423976264|0.634764556|Rockledge, Florida|
+------------------+----------------+-----------+-----------+------------------+
only showing top 5 rows



In [4]:
# load songs
song_df = spark.read.options(sep='\t', header=False, inferSchema=True).csv('./data/songs.tsv')
song_df = song_df.toDF('id', 
                       'title', 
                       'year',
                       'hotness',
                       'id_artist',
                       'id_album',
                       'duration',
                       'end_of_fade_on',
                       'start_of_fade_out',
                       'tempo',
                       'time_signature', 
                       'key', 
                       'loudness', 
                       'mode', 
                       'style')
song_df.createOrReplaceTempView('song')
spark.sql('SELECT * FROM song').show(5)

+------------------+--------------------+----+-----------+------------------+--------+---------+--------------+-----------------+-------+--------------+---+--------+----+-----------------+
|                id|               title|year|    hotness|         id_artist|id_album| duration|end_of_fade_on|start_of_fade_out|  tempo|time_signature|key|loudness|mode|            style|
+------------------+--------------------+----+-----------+------------------+--------+---------+--------------+-----------------+-------+--------------+---+--------+----+-----------------+
|SOAAAQN12AB01856D3|Campeones De La Vida|   0|0.427446572|ARAMIDF1187FB3D8D4|  346208|153.36444|         0.235|          142.518|120.051|             4|  4|   -8.76|   0|         pop rock|
|SOAABSU12A81C1FB9E|              Lonely|   0|         NA|ARVUVRE12420780B0B|  230893|240.29995|         0.166|          226.482|166.124|             4|  1|  -6.735|   1|        melbourne|
|SOAAEHR12A6D4FB060| Slaves & Bulldozers|1991|0.6540679

### 1.¿Cuál es el estilo más lento(tempo) en media?

In [5]:
spark.sql('''
    SELECT style, mean(tempo) tempo
    FROM song
    GROUP BY style
    ORDER BY tempo
''').show(1)

+--------+------+
|   style| tempo|
+--------+------+
|rebetika|47.447|
+--------+------+
only showing top 1 row



### 2.¿Cuales  son  los  5  artistas,  ubicados  en  UK(cualquier  territorio  de  UK),  con mayor número de canciones en escala menor (mode = 1)?

In [6]:
spark.sql('''
    SELECT name, 
        --a.location, 
        count(*) n_songs
    FROM artist a, song s
    WHERE a.id = s.id_artist
        AND (   a.location LIKE '%UK%' OR 
                a.location LIKE '%United Kingdom%' OR 
                a.location LIKE '%England%' OR 
                a.location LIKE '%Wales%' OR
                a.location LIKE '%Scotland%' OR
                a.location LIKE '%Northern Ireland%')
        AND s.mode == 1
    GROUP BY a.name--, a.location
    ORDER BY n_songs DESC
''').show(6)

+------------------+-------+
|              name|n_songs|
+------------------+-------+
|        SNOWPATROL|     11|
|      Phil Collins|      9|
|         Radiohead|      7|
|The Rolling Stones|      7|
|       Bad Company|      6|
|              Seal|      6|
+------------------+-------+
only showing top 6 rows



### 3.Desde  1970  hasta  hoy,  ¿las  canciones  son  más  rápidas  (tempo),  altas (loudness)  y cortas  (duration)  en  media?  Ordena  los  resultados  por  año ascendente.

In [7]:
spark.sql('''
    SELECT year, mean(tempo) tempo, mean(loudness) loudness, mean(duration) duration
    FROM song s
    WHERE year >= 1970
    GROUP BY year
    ORDER BY year
''').show()

+----+------------------+-------------------+------------------+
|year|             tempo|           loudness|          duration|
+----+------------------+-------------------+------------------+
|1970|121.34628571428576| -11.92847619047619|231.42578619047612|
|1971|136.16195999999997|-12.153000000000002|259.55428919999997|
|1972|129.17204166666667|-11.719291666666665|238.54539749999995|
|1973|        116.356125|-11.711541666666664|294.16444416666667|
|1974|125.08609090909091|-10.670681818181817|239.49134636363632|
|1975|125.41183333333332|-11.249541666666666| 277.4406354166666|
|1976|137.26139999999998|           -11.6584|210.99404933333338|
|1977|139.33685714285716|-11.820114285714288|255.30692799999997|
|1978|         134.38385|           -10.1125|247.85456749999997|
|1979|137.51694444444445|-11.879083333333332| 226.0566886111111|
|1980|126.89337499999999|-11.098531250000002|     210.438730625|
|1981|127.96074999999999| -11.57044444444444|211.69224499999987|
|1982|         125.14522|

### 4.¿Cuál es el estilo que menos abusa de los efectos de fade in y fade out (mayor número de segundos desde inicio al final del fade in más desde el inicio del fade out al final de la canción?

In [8]:
spark.sql('''
    SELECT style,
           mean((end_of_fade_on + (duration - start_of_fade_out))) fade
    FROM song
    GROUP BY style
    ORDER BY fade 
''').show(5)

+------------------+--------------------+
|             style|                fade|
+------------------+--------------------+
|    power violence|-3.29999999998165...|
|christian hardcore|-1.95000000005052...|
|east coast hip hop|1.199999999812462...|
|    argentine rock|5.099999999913507E-4|
|    melodic trance|               0.103|
+------------------+--------------------+
only showing top 5 rows



Comprobamos que tengan sentido resultados negativos (serán datos mal guardados.)

In [9]:
spark.sql('''
    SELECT title,
           end_of_fade_on,
           duration, 
           start_of_fade_out,
           (end_of_fade_on + (duration - start_of_fade_out)) fade
    FROM song
    WHERE style = 'power violence' OR style = 'christian hardcore'
    ORDER BY fade 
''').toPandas()

,title,end_of_fade_on,duration,start_of_fade_out,fade
0,Inherent Scars,0.0,245.13261,245.133,-0.00039
1,Wepons Of Mass Deception (Album),0.0,49.94567,49.946,-0.00033
2,I Versus I,0.0,184.84200,184.842,0.00000


### 5.¿Cual es la canción menos popular (hotness) de los 5 artistas más populares (hotness)?

In [10]:
spark.sql('''
    SELECT title, hotness FROM (
        SELECT title, hotness,
        RANK() OVER (PARTITION BY id_artist ORDER BY hotness) rank
        FROM song
        WHERE id_artist in (
            SELECT id
            FROM artist
            ORDER BY hotness DESC
            LIMIT 5
        )
    )
    WHERE rank = 1
''').show(5)

+--------------------+-----------+
|               title|    hotness|
+--------------------+-----------+
|             Da Funk|  0.8622545|
|Speed Of Sound (L...|0.454042766|
|Skit #2 (Kanye We...|  0.7801197|
| The Way I Loved You|0.853828893|
|               Magic|0.508602172|
+--------------------+-----------+



In [11]:
spark.stop()